In [ ]:
!pip install transformers[sentencepiece]
!pip install datasets
!pip install evaluate
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 35.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 31.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 61.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

In [ ]:
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, DataCollatorWithPadding, TrainingArguments, Trainer, AutoModel
from datasets import Dataset
import evaluate

Загружаем отобранные предложения. Предложения отбираются в скрипте precorrect.py

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
with open('/content/drive/My Drive/CorrectDN.json', encoding="utf-8") as f:
#    sentences = pd.read_json(f, orient='records')
#entences.tail()
    novanews = pd.read_json(f, orient='records')
novanews.tail()

,text,label
5378,В Москве открылись избирательные участки на вы...,1
5379,Трое детей погибли при обстреле Донецка. ДОНЕЦ...,1
5380,В Липецкой области нейтрализовали два беспилот...,1
5381,Болгария с апреля начнет выдавать россиянам ше...,1
5382,Все участки на выборах президента в Москве отк...,1


In [ ]:
with open('/content/drive/My Drive/editorriaDN.json', encoding="utf-8") as f:
    gitria = pd.read_json(f, orient='records')
gitria.tail()
#    sentences = pd.read_json(f, orient='records')
# sentences.tail()

,text,label
3995,"Головная компания группы ""Тинькофф"" ушла из ки...",1929478622
3996,"В ""Моспроекте"" продали долю основателя S-Holdi...",1929478546
3997,"В ""Сириусе"" завершается подготовка объектов Вс...",1929477450
3998,"В Германии рассказали, когда у ВСУ кончатся сн...",1929477293
3999,"""Опасная практика"". В ICAN осудили размещение ...",1929476595


In [ ]:
frames = [novanews, gitria]
sentences = pd.concat(frames)

Собираем dataset

In [ ]:
raw_ds = Dataset.from_pandas(sentences, preserve_index=False)
raw_ds = raw_ds.class_encode_column('label')
raw_ds.features

Stringifying the column:   0%|          | 0/9383 [00:00<?, ? examples/s]

Casting to class labels:   0%|          | 0/9383 [00:00<?, ? examples/s]

{'text': Value(dtype='string', id=None),
 'label': ClassLabel(names=['1', '1599002923', '1599794219', '1599982658', '1600141843', '1600509253', '1600793639', '168438071', '168709136', '1729542270', '1730765373', '1870201378', '1879513265', '1908139109', '1915598141', '1915600208', '1915630213', '1916753876', '1916777953', '1917284371', '1925000256', '1925459242', '1925779902', '1927795387', '1928461921', '1928743846', '1928918545', '1928933663', '1928937522', '1929009548', '1929025321', '1929055296', '1929066530', '1929433864', '1929476595', '1929477293', '1929477450', '1929478546', '1929478622', '1929478838', '1929479456', '1929479631', '1929480299', '1929480493', '1929480858', '1929480983', '1929481119', '1929481249', '1929481418', '1929481685', '1929481882', '1929482308', '1929482452', '1929482548', '1929482694', '1929482839', '1929482998', '1929483156', '1929484651', '1929485194', '1929485358', '1929485490', '1929485681', '1929485961', '1929486583', '1929486783', '1929486894', '192

Делим dataset на train и test (80:20)

In [ ]:
raw_ds = raw_ds.train_test_split(test_size=0.2, shuffle=True)

In [ ]:
from transformers import AutoModelForMaskedLM, AutoTokenizer

model_checkpoint = "ai-forever/ruRoberta-large"
# model_checkpoint = "/content/drive/My Drive/fine-train-ruroberta202403221e4"
model = AutoModelForMaskedLM.from_pretrained(model_checkpoint)
# model = AutoModelForMaskedLM.from_pretrained("/content/drive/My Drive/fine-train")
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/674 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.81M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.37M [00:00<?, ?B/s]

Tokenized dataset

In [ ]:
def tokenize_function(examples):
    result = tokenizer(examples["text"])
    if tokenizer.is_fast:
        result["word_ids"] = [result.word_ids(i) for i in range(len(result["input_ids"]))]
    return result


# Use batched=True to activate fast multithreading!
tokenized_datasets = raw_ds.map(
    tokenize_function, batched=True, remove_columns=["text", "label"]
)
# tokenized_datasets

Map:   0%|          | 0/7506 [00:00<?, ? examples/s]

Map:   0%|          | 0/1877 [00:00<?, ? examples/s]

Chank size (Parameter!!!)

In [ ]:
chunk_size = 256

In [ ]:
def group_texts(examples):
    # Concatenate all texts
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    # Compute length of concatenated texts
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the last chunk if it's smaller than chunk_size
    total_length = (total_length // chunk_size) * chunk_size
    # Split by chunks of max_len
    result = {
        k: [t[i : i + chunk_size] for i in range(0, total_length, chunk_size)]
        for k, t in concatenated_examples.items()
    }
    # Create a new labels column
    result["labels"] = result["input_ids"].copy()
    return result

In [ ]:
lm_datasets = tokenized_datasets.map(group_texts, batched=True)
# lm_datasets

Map:   0%|          | 0/7506 [00:00<?, ? examples/s]

Map:   0%|          | 0/1877 [00:00<?, ? examples/s]

Процент маскирования = 15% (Parameter!!!) Оптимизировано!

In [ ]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

**Не используется!**<br>
whole word masking, процент маскирования = 15% (Parameter!!!)

In [ ]:
import collections
import numpy as np

from transformers import default_data_collator

wwm_probability = 0.15


def whole_word_masking_data_collator(features):
    for feature in features:
        word_ids = feature.pop("word_ids")

        # Create a map between words and corresponding token indices
        mapping = collections.defaultdict(list)
        current_word_index = -1
        current_word = None
        for idx, word_id in enumerate(word_ids):
            if word_id is not None:
                if word_id != current_word:
                    current_word = word_id
                    current_word_index += 1
                mapping[current_word_index].append(idx)

        # Randomly mask words
        mask = np.random.binomial(1, wwm_probability, (len(mapping),))
        input_ids = feature["input_ids"]
        labels = feature["labels"]
        new_labels = [-100] * len(labels)
        for word_id in np.where(mask)[0]:
            word_id = word_id.item()
            for idx in mapping[word_id]:
                new_labels[idx] = labels[idx]
                input_ids[idx] = tokenizer.mask_token_id
        feature["labels"] = new_labels

    return default_data_collator(features)

Маскирование всего набора за раз

In [ ]:
def insert_random_mask(batch):
    features = [dict(zip(batch, t)) for t in zip(*batch.values())]
    masked_inputs = data_collator(features)
    # Create a new "masked" column for each column in the dataset
    return {"masked_" + k: v.numpy() for k, v in masked_inputs.items()}

In [ ]:
lm_datasets = lm_datasets.remove_columns(["word_ids"])
eval_dataset = lm_datasets["test"].map(
    insert_random_mask,
    batched=True,
    remove_columns=lm_datasets["test"].column_names,
)
eval_dataset = eval_dataset.rename_columns(
    {
        "masked_input_ids": "input_ids",
        "masked_attention_mask": "attention_mask",
        "masked_labels": "labels"
    }
)

Map:   0%|          | 0/2192 [00:00<?, ? examples/s]

Batch_size = **8** (Parameter!!!)

In [ ]:
from torch.utils.data import DataLoader
from transformers import default_data_collator

batch_size = 8
train_dataloader = DataLoader(
    lm_datasets["train"],
    shuffle=True,
    batch_size=batch_size,
    collate_fn=data_collator,
    # collate_fn=whole_word_masking_data_collator
)
eval_dataloader = DataLoader(
    eval_dataset, batch_size=batch_size, collate_fn=default_data_collator
)

Optimizer = 1e-4 (Parameter!!!)

In [ ]:
from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr=1e-4)

In [ ]:
from accelerate import Accelerator

accelerator = Accelerator()
model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(
    model, optimizer, train_dataloader, eval_dataloader
)

In [ ]:
from transformers import get_scheduler

num_train_epochs = 5
num_update_steps_per_epoch = len(train_dataloader)
num_training_steps = num_train_epochs * num_update_steps_per_epoch

lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

In [ ]:
output_dir = '/content/drive/My Drive/fine-train'


In [ ]:
from tqdm.auto import tqdm
import torch
import math

progress_bar = tqdm(range(num_training_steps))

for epoch in range(num_train_epochs):
    # Training
    model.train()
    for batch in train_dataloader:
        outputs = model(**batch)
        loss = outputs.loss
        accelerator.backward(loss)

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

    # Evaluation
    model.eval()
    losses = []
    for step, batch in enumerate(eval_dataloader):
        with torch.no_grad():
            outputs = model(**batch)

        loss = outputs.loss
        losses.append(accelerator.gather(loss.repeat(batch_size)))

    losses = torch.cat(losses)
    losses = losses[: len(eval_dataset)]
    try:
        perplexity = math.exp(torch.mean(losses))
    except OverflowError:
        perplexity = float("inf")

    print(f">>> Epoch {epoch}: Perplexity: {perplexity}")

    # Save and upload
    accelerator.wait_for_everyone()
    unwrapped_model = accelerator.unwrap_model(model)
    unwrapped_model.save_pretrained(output_dir, save_function=accelerator.save)
    if accelerator.is_main_process:
        tokenizer.save_pretrained(output_dir)

  0%|          | 0/5620 [00:00<?, ?it/s]

>>> Epoch 0: Perplexity: 3692.309336108749
>>> Epoch 1: Perplexity: 3322.8043092295984
>>> Epoch 2: Perplexity: 3290.3435257100223
>>> Epoch 3: Perplexity: 3271.435634571224
>>> Epoch 4: Perplexity: 3254.5694899034124
